In [30]:
import pandas as pd
import numpy as np
import os

In [31]:
def get_station_meta(station_number,file_prefix='hly'):
    station_meta = dict()
    station_header_file = f'data/{file_prefix}{station_number}_header.csv'
    df_header = pd.read_csv(station_header_file)
    for row in df_header[df_header.columns[0]]:
        if 'Latitude' in row:
            split_vals = row.split(',')
            station_meta['latitude'] = float(split_vals[0].replace('Latitude:',''))
            station_meta['longitude'] = float(split_vals[-1].replace('Longitude:',''))
    return station_meta

In [88]:
raw_data_files = [file for file in os.listdir('data/') if 'gz.parquet' in file and ('hly' in file or 'dly' in file)]
#raw_data_files = list(np.random.choice(raw_data_files,15))
#raw_data_files

In [89]:
df_processed = pd.DataFrame()
for file in raw_data_files:
    station_number = int(file.replace('.gz','').replace('.parquet','').replace('hly','').replace('dly',''))
    station_file = f'data/{file}'
    file_prefix = file[0:3]
    station_meta = get_station_meta(station_number,file_prefix)
    print(station_file)
    df = pd.read_parquet(station_file)
    print(df.shape)
    # only process data if we have some
    if len(df)>0:
        df = df.set_index(pd.to_datetime(df.index))
        df.columns = [f'{file_prefix}_{col}' for col in df.columns.values]
        df = df.set_index(df.index.to_series().dt.date)
        if file_prefix == 'dly':
            df = df.groupby(by=df.index).agg([np.mean])
        elif file_prefix == 'hly':
            df = df.groupby(by=df.index).agg([np.mean, np.min, np.max, np.std])
        df.columns = ['_'.join(col).strip() for col in df.columns.values]
        df = df.dropna(how='all')
        df = df.round(2)
        df['station_number'] = station_number
        df['latitude'] = station_meta['latitude']
        df['longitude'] = station_meta['longitude']
        print(df.shape)
        df_processed = df_processed.append(df,sort=True)
        print(df_processed.shape)
print(df_processed.shape)
df_processed = df_processed.groupby([df_processed.index,'station_number']).max()
print(df_processed.shape)
df_processed.to_parquet('data/daily_by_station.gz.parquet',compression='gzip')
df_processed.head()

data/dly1024.gz.parquet
(27426, 2)
(27426, 5)
(27426, 5)
data/dly1042.gz.parquet
(24136, 2)
(24136, 5)
(51562, 5)
data/dly1043.gz.parquet
(16985, 2)
(16985, 5)
(68547, 5)
data/dly1075.gz.parquet
(19455, 23)
(19455, 26)
(88002, 26)
data/dly108.gz.parquet
(28579, 2)
(28579, 5)
(116581, 26)
data/dly1103.gz.parquet
(22767, 2)
(22767, 5)
(139348, 26)
data/dly1107.gz.parquet
(13968, 2)
(13968, 5)
(153316, 26)
data/dly1108.gz.parquet
(7274, 2)
(7274, 5)
(160590, 26)
data/dly1116.gz.parquet
(12722, 8)
(12722, 11)
(173312, 28)
data/dly1117.gz.parquet
(13483, 2)
(13483, 5)
(186795, 28)
data/dly1128.gz.parquet
(25873, 2)
(25873, 5)
(212668, 28)
data/dly1130.gz.parquet
(20512, 2)
(20512, 5)
(233180, 28)
data/dly1143.gz.parquet
(16496, 2)
(16496, 5)
(249676, 28)
data/dly1208.gz.parquet
(7578, 2)
(7578, 5)
(257254, 28)
data/dly1209.gz.parquet
(14000, 2)
(14000, 5)
(271254, 28)
data/dly1225.gz.parquet
(21396, 2)
(21396, 5)
(292650, 28)
data/dly1233.gz.parquet
(12630, 2)
(12630, 5)
(305280, 28)
data/d

dly_cbl_mean  dly_ddhm_mean  dly_dos_mean  \
date       station_number                                              
1941-01-01 108                      NaN            NaN           NaN   
           201                      NaN            NaN           NaN   
           405                      NaN            NaN           NaN   
           417                      NaN            NaN           NaN   
           418                      NaN            NaN           NaN   

                           dly_evap_mean  dly_g_rad_mean  dly_glorad_mean  \
date       station_number                                                   
1941-01-01 108                       NaN             NaN              NaN   
           201                       NaN             NaN              NaN   
           405                       NaN             NaN              NaN   
           417                       NaN             NaN              NaN   
           418                       NaN             NaN              NaN   

                           dly_gmin_mean  dly_hg_mean  dly_hm_mean  \
date       station_number                                            
1941-01-01 108                       NaN          NaN          NaN   
           201                       NaN          NaN          NaN   
           405                       NaN          NaN          NaN   
           417                       NaN          NaN          NaN   
           418                       NaN          NaN          NaN   

                           dly_igmin_mean  ...  hly_wetb_amax  hly_wetb_amin  \
date       station_number                  ...                                 
1941-01-01 108                        NaN  ...            NaN            NaN   
           201                        NaN  ...            NaN            NaN   
           405                        NaN  ...            NaN            NaN   
           417                        NaN  ...            NaN            NaN   
           418                        NaN  ...            NaN            NaN   

                           hly_wetb_mean  hly_wetb_std  hly_ww_amax  \
date       station_number                                             
1941-01-01 108                       NaN           NaN          NaN   
           201                       NaN           NaN          NaN   
           405                       NaN           NaN          NaN   
           417                       NaN           NaN          NaN   
           418                       NaN           NaN          NaN   

                           hly_ww_amin  hly_ww_mean  hly_ww_std  latitude  \
date       station_number                                                   
1941-01-01 108                     NaN          NaN         NaN    52.311   
           201                     NaN          NaN         NaN    51.735   
           405                     NaN          NaN         NaN    51.978   
           417                     NaN          NaN         NaN    52.842   
           418                     NaN          NaN         NaN    52.780   

                           longitude  
date       station_number             
1941-01-01 108                -6.766  
           201                -9.546  
           405                -9.622  
           417                -9.238  
           418                -8.903  

[5 rows x 110 columns]

In [64]:
#df_processed.index.to_series().dt.date